In [1]:
!python -V

Python 3.9.19


In [78]:
import urllib.request
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [33]:
def load_data(list_file_name: list):
    for file_name in list_file_name:
        # check if file already exist
        if not os.path.exists(f"./data/{file_name}"): # Or folder, will return true or false
            if not os.path.exists("./data/"):
                os.mkdir("./data/")
            site_url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file_name}"
            file_name = f"./data/{file_name}"
            urllib.request.urlretrieve(site_url, filename=file_name)
            print(f"load file: {file_name}")
        else:
            print(f'file {file_name} already exist')

In [34]:
list_of_file = ["yellow_tripdata_2023-01.parquet", "yellow_tripdata_2023-02.parquet"]
load_data(list_of_file)

file yellow_tripdata_2023-01.parquet already exist
file yellow_tripdata_2023-02.parquet already exist


## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19 (x)

In [35]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
print(f"columns count: {len(df.columns)}")

columns count: 19


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59 (x)
- 52.59
- 62.59

In [58]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print(f"std of duration is {df['duration'].std()} mins")

std of duration is 42.59435124195458 mins


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98% (x)

In [53]:
df_drop = df[(df.duration >= 1) & (df.duration <= 60)]
record_left = (len(df_drop)/ len(df)) * 100
print(f" fraction of the records left = {record_left}")

 fraction of the records left = 98.1220282212598


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515 (x)
- 715

In [85]:
# Specify the categorical columns
categorical = ['PULocationID', 'DOLocationID']
# Convert categorical columns to strings
df_drop[categorical] = df_drop[categorical].astype(str)
# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
train_dicts = df_drop[categorical].to_dict(orient='records')
# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
# Get a feature matrix from it
print(f"dimensionality of this matrix: {X_train.shape}")

C:\Users\Booklyn\AppData\Local\Temp\ipykernel_13476\2496641530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[categorical] = df_drop[categorical].astype(str)


dimensionality of this matrix: (3009173, 515)


## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 3.64
- 7.64 (x)
- 11.64
- 16.64

In [86]:
target = 'duration'
y_train = df_drop[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [87]:
print(f"rmse: {mean_squared_error(y_train, y_pred, squared=False)}")

rmse: 7.6492611527240895


c:\Users\Booklyn\miniconda3\envs\mlops\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81 (x)
- 11.81
- 16.81

In [104]:
def read_df(file_name):
    df = pd.read_parquet(file_name)
    # transformed
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    # Specify the categorical columns
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [106]:
df_train = read_df('./data/yellow_tripdata_2023-01.parquet')
df_val = read_df('./data/yellow_tripdata_2023-02.parquet')

categorical = ['PULocationID', 'DOLocationID']
dv = DictVectorizer()
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

In [107]:
print(f"rmse: {mean_squared_error(y_val, y_pred, squared=False)}")


rmse: 7.811817646307258


c:\Users\Booklyn\miniconda3\envs\mlops\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
